In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [3]:
noise = np.load('../../The-Quest-For-Mu/noise200_200.npy')

In [4]:
artificial_tracks = np.load('../../The-Quest-For-Mu/convolves2.npy')

In [146]:
def gen_big_pictures_dataset(tracks, noise, size, tracks_count, shape=(200, 200)):
    assert(len(shape) == 2)
    assert(shape[0] == shape[1])
    
    dataset = np.zeros(shape[0] * shape[1])
    targets = np.zeros(shape[0] * shape[1])
    
    for j in xrange(size):
        print j
        template = np.zeros(shape)
        answer = np.zeros(shape)
        noise_template = noise[np.random.choice(np.arange(noise.shape[0]))].reshape(200, 200)
        signal_samples = artificial_tracks[np.random.choice(np.arange(artificial_tracks.shape[0]),
                                                            replace=False,
                                                            size=tracks_count)]
        
        if tracks_count > 0:
            tlx = np.random.choice(np.arange(shape[0] - signal_samples[0].shape[0]), replace=False, size=tracks_count)
            tly = np.random.choice(np.arange(shape[1] - signal_samples[0].shape[1]), replace=False, size=tracks_count)
        
        for i in xrange(tracks_count):
            answer[tlx[i]:tlx[i] + signal_samples[0].shape[0],\
                   tly[i]:tly[i] + signal_samples[0].shape[1]] = 1
            template[tlx[i]:tlx[i] + signal_samples[0].shape[0],\
                     tly[i]:tly[i] + signal_samples[0].shape[1]] = signal_samples[i]
                
        
        sample = np.max([5 * template / (1e-6 + template.max() - template.min()),
                         6 * noise_template / (1e-6 + noise_template.max() - noise_template.min())],
                        axis=0)
        
        dataset = np.vstack((dataset, sample.ravel()))
        targets = np.vstack((targets, answer.ravel()))
                
    return dataset, targets

In [147]:
%time
dataset, targets = gen_big_pictures_dataset(artificial_tracks, noise, 1000, 3)

CPU times: user 7 µs, sys: 4 µs, total: 11 µs
Wall time: 22.9 µs
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
2

In [150]:
dataset = dataset[1:]
targets = targets[1:]

In [ ]:
import lasagne
import theano
import theano.tensor as T
import numpy as np
from lasagne.layers import DenseLayer,\
                           Conv2DLayer,\
                           MaxPool2DLayer,\
                           InputLayer,\
                           GlobalPoolLayer,\
                           DropoutLayer,\
                           Upscale2DLayer
from lasagne.nonlinearities import elu, sigmoid, rectify


class Base(object):
    def __init__(self, img_shape=(200, 200), learning_rate=1e-3):
        self.output_layer = self.build_network(img_shape)
        self.train_network = self.compile_trainer(learning_rate)
        self.eval_network = self.compile_evaluator()
        self.forward_pass = self.compile_forward_pass()


    # TODO: could be modified
    # add cost as sum nonzeros elements in the middle layers
    def get_obj(self, input_X, targets, c=1e-3):
        answers = lasagne.layers.get_output(self.output_layer, input_X)
        
        obj = (targets * T.log(targets * answers)).sum() +\
          c * ((1 - targets) * T.log(1 - (1 - targets) * answers)).sum()

        return obj
    
    def get_accuracy(self, input_X, targets):
        predictions = lasagne.layers.get_output(self.output_layer, input_X)
        
        return lasagne.objectives.binary_accuracy(predictions.ravel(), targets.ravel()).mean()

    # TODO: could be modified
    def compile_trainer(self, learning_rate):
        targets = T.tensor4('targets')
        input_X = T.tensor4('inputs')

        answers = lasagne.layers.get_output(self.output_layer, input_X)

        obj = self.get_obj(input_X, targets)

        params = lasagne.layers.get_all_params(self.output_layer, trainable=True)

        updates = lasagne.updates.adamax(obj,
                                         params,
                                         learning_rate=learning_rate)
        return theano.function([targets, input_X], obj, updates=updates)

    # TODO: could be modified
    def build_network(self, img_shape):
        input_layer = InputLayer((None, 1) + tuple(img_shape),
                                  name='network input')

        l1 = Conv2DLayer(input_layer,
                         nonlinearity=rectify,
                         num_filters=6,
                         filter_size=(3, 3),
                         pad='same',
                         name='l1 conv')
        mp12 = MaxPool2DLayer(l1, pool_size=2, name='l12 max_pool')
        l2 = Conv2DLayer(mp12,
                         nonlinearity=rectify,
                         num_filters=12,
                         filter_size=(3, 3),
                         pad='same',
                         name='l2 conv')
        mp2 = MaxPool2DLayer(l2, pool_size=10, name='l12 max_pool')

        return l2
    
    def build_transform_network(self):
        net = InputLayer((None, 12, 1))
        net = DenseLayer(net, num_units=1024)
        net = DenseLayer(net, num_units=1, nonlinearity=sigmoid)
        
        return net

    # TODO: could be modified
    def compile_forward_pass(self):
        input_X =  T.tensor4('input images')

        res = lasagne.layers.get_output(self.output_layer, input_X)

        return theano.function([input_X], res)

    def compile_evaluator(self):
        targets = T.tensor4('targets')
        input_X = T.tensor4('inputs')

        obj = self.get_obj(input_X, targets)

        return theano.function([targets, answers], self.get_accuracy(input_X, targets))


In [159]:
def iterate_minibatches(inputs, targets, batch_size):
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        excerpt = indices[start_idx:start_idx + batch_size]

        yield inputs[excerpt], targets[excerpt]

In [160]:
def learn(nn, dataset, num_epochs=100, batch_size=50):
    '''
        num_epochs - amount of passes through the data
        batch_size - number of samples processed at each function call
        dataset - [X_train, X_val, y_train, y_val]
    '''
    
    X_train, X_val, y_train, y_val = dataset

    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_acc = 0
        train_batches = 0
        start_time = time.time()

        for batch in iterate_minibatches(X_train, y_train, batch_size):
            inputs, targets = batch
            train_err_batch, train_acc_batch = nn.train(inputs, targets)
            train_err += train_err_batch
            train_acc += train_acc_batch
            train_batches += 1

        # And a full pass over the validation data:
        val_acc = 0
        val_batches = 0

        for batch in iterate_minibatches(X_val, y_val, batch_size):
            inputs, targets = batch
            val_acc += nn.evaluate(inputs, targets)
            val_batches += 1


        # Then we print the results for this epoch:
        print('Epoch {} of {} took {:.3f}s'.format(epoch + 1, num_epochs, time.time() - start_time))
        print('  training loss (in-iteration):\t\t{:.6f}'.format(train_err / train_batches))
        print('  train accuracy:\t\t{:.2f} %'.format(train_acc / train_batches * 100))
        print('  validation accuracy:\t\t{:.2f} %'.format(val_acc / val_batches * 100))